### OpenAI vs HuggingFace ###

In [ ]:
from langchain_openai import OpenAI


In [ ]:
from dotenv import load_dotenv

load_dotenv()


In [ ]:
llm = OpenAI(temperature=0.7)

In [ ]:
text = 'What is the capital of Pakistan'
print(llm.invoke(text))

In [ ]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"]=""

In [ ]:
from langchain import HuggingFaceHub

In [ ]:
llm_huggingface = HuggingFaceHub(repo_id="google/flan-t5-large", model_kwargs={"temperature":0, "max_length":64})

In [ ]:
output = llm_huggingface.predict("Can you tell me the capital of Russia")
print(output)

In [ ]:
output = llm_huggingface.predict("Can you tell me the capital of Pakistan")
print(output)

In [ ]:
output = llm_huggingface.predict("Can you write a poen about AI")
print(output)

In [ ]:
output = llm.predict("Can you write a poen about AI")
print(output)

### Prompt Template And LLMChain ###

In [ ]:
from langchain.prompts import PromptTemplate

In [ ]:
prompt_template = PromptTemplate(input_variables=['country'],
                                 template="Tell me the capital of this {country}")

In [ ]:
prompt_template.format(country = "India")

In [ ]:
from langchain.chains import LLMChain

In [ ]:
chains = LLMChain(llm = llm, prompt = prompt_template)
chains.run("india")

### Combining Multiple Chains ###

In [ ]:
capital_template = PromptTemplate(input_variables=['country'],
                                template = "Please tell me the capital of thr {country}")

capital_chain = LLMChain(llm = llm, prompt = capital_template)

famous_template = PromptTemplate(input_variables=['capital'],
                                 template="Suggest me some amazing places to visit in {capital}")

famous_chain = LLMChain(llm=llm, prompt = famous_template)

In [ ]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains=[capital_chain, famous_chain])
chain.run("Pakistan")

### Sequential Chain ###

In [ ]:
capital_template = PromptTemplate(input_variables=['country'],
                                  template = "Please tell me the capital of the {country}")
capital_chain = LLMChain(llm=llm, prompt = capital_template, output_key = "capital")

In [ ]:
famous_template = PromptTemplate(input_variables=['capital'],
                                 template="Suggest me some amazing places to visit in {capital}")

famous_chain = LLMChain(llm=llm, prompt = famous_template, output_key="places")

In [ ]:
from langchain.chains import SequentialChain
chain = SequentialChain(chains = [capital_chain, famous_chain],
                        input_variables = ['country'],
                        output_variables = ['capital','places'])

In [ ]:
chain({'country': 'Pakistan'})

### ChatModels with ChatOpenAI

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

In [ ]:
import os
os.environ["OPEN_API_KEY"]=""

In [ ]:
chatllm = ChatOpenAI(openai_api_key = os.environ["OPEN_API_KEY"], temperature=0.6, model = 'gpt-3.5-turbo')
chatllm

In [ ]:
chatllm([
    SystemMessage(content="You are a comedian AI assistant"),
    HumanMessage(content="Please provide some comedy punchlines on AI")
])

### Prompt Template + LLM + Output Parsers ###

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [72]:
class Commaseparatedoutput(BaseOutputParser):
    def parse(self, text:str):
        return text.strip().split(",")

In [73]:
template = "You are a helpful assistant. When the user given any input, you should generate 5 words that should be comma separated"
human_template = "{text}"
chatprompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template)
])

In [74]:
chain = chatprompt | chatllm | Commaseparatedoutput()

In [77]:
chain.invoke({"text":"intelligent"})

['smart', ' clever', ' bright', ' astute', ' knowledgeable']